## 메모리를 체인에 꽂기 
## 두 종류의 체인을 사용해서 꽂기
# 1. llm chian : off-the-shelf : 일반적인 목적을 가진 체인
# 2. custom chain

In [12]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

template = """
    You are a helpful AI talking to a human.
    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True ## 로그 확인용
)

chain.predict(question="my name is jm")
chain.predict(question="i live in yong in")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human:my name is jm
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    Human: my name is jm
AI: Nice to meet you, JM! How can I assist you today?
    Human:i live in yong in
    You:


> Finished chain.


"That's great to hear! How can I assist you with living in Yongin today?"

In [15]:
chain.predict(question="what is my name")
## 메모리, history를 프롬프트에 넣지 않으면 대화를 까먹음



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    Human: my name is jm
AI: Nice to meet you, JM! How can I assist you today?
Human: i live in yong in
AI: That's great to hear! How can I assist you with living in Yongin today?
Human: what is my name
AI: Your name is JM.
Human: what is my name
AI: Your name is JM.
    Human:what is my name
    You:


> Finished chain.


'Your name is JM.'

In [16]:
memory.load_memory_variables({})
## 실행해보면 위에 llmchian에 설정으로 memory 사용한다고 해서 메모리에는 남고 있으나, 
## 프롬프트에 함께 넣지 않아서 기억?을 하지 못하는 것

{'chat_history': 'System: The human introduces themselves as "jm".\nAI: Nice to meet you, JM! How can I assist you today?\nHuman: i live in yong in\nAI: That\'s great to hear! How can I assist you with living in Yongin today?\nHuman: what is my name\nAI: Your name is JM.\nHuman: what is my name\nAI: Your name is JM.\nHuman: what is my name\nAI: Your name is JM.'}

## CHAT BASED MEMORY
대화 기반의 메세지 메모리
2 가지 방식
1) 문자열 
2) 메세지

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True ## 문자열로 바꾸는게 아니라 실제 message 클래스로 변경
)

prmopt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful ai talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"), ## 메모리 클래스로 대체
    ("human", "{question}")
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prmopt,
    verbose=True,
)

chain.predict(question="My name is JM")
